In [ ]:
!pip install -q efficientnet

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import math

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import Sequence
import tensorflow.keras.layers as L
import efficientnet.tfkeras as efn

In [ ]:
# limit the GPU memory growth
gpu = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu))
if len(gpu) > 0:
    tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
# detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
data_dir = Path('../input/seti-breakthrough-listen/')
train_data_dir = data_dir / 'train'
test_data_dir = data_dir / 'test'

train_label_file = data_dir / 'train_labels.csv'
sample_file = data_dir / 'sample_submission.csv'

In [ ]:
id_col = 'id'
target_col = 'target'

label = pd.read_csv(train_label_file, index_col=id_col)
sub = pd.read_csv(sample_file, index_col=id_col)

In [ ]:
label['target'].value_counts()

In [ ]:
# sample_df = label.sample(frac=1).reset_index(drop=True)
sample_df_0 = label[label['target'] == 0]
sample_df_0_reduced = sample_df_0.sample(n=4694)

sample_df_1 = label[label['target'] == 1]

new_label = pd.concat([sample_df_1, sample_df_0_reduced])

new_label['target'].value_counts()

In [ ]:
def id_to_path(s, train=True):
    data_dir = train_data_dir if train else test_data_dir
    return data_dir / s[0] / f'{s}.npy'

In [ ]:
input_size = (273, 256, 3)
batch_size = 32
n_epoch = 5
seed = 42

In [ ]:
class SETISequence(Sequence):
    def __init__(self, x_set, y_set=None, batch_size=32):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.is_train = False if y_set is None else True
    
    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    
    def __getitem__(self, idx):
        batch_ids = self.x[idx * self.batch_size: (idx + 1) * self.batch_size]
        if self.y is not None:
            batch_y = self.y[idx * self.batch_size: (idx + 1) * self.batch_size]
        
        # taking channels 1, 3, and 5 only
        list_x = [np.load(id_to_path(x, self.is_train))[::2] for x in batch_ids]
        batch_x = np.moveaxis(list_x,1,-1)
        batch_x = batch_x.astype("float") / 255
        
        if self.is_train:
            return batch_x, batch_y
        else:
            return batch_x

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

# Completed Dataset

In [ ]:
x0 = label.index.values
y0 = label[target_col].values

x1 = sub.index.values

x_train, x_val, y_train, y_val = train_test_split(x0, y0, test_size=.2, random_state=seed)

train_complete = SETISequence(x_train, y_train, batch_size=batch_size)
val_complete = SETISequence(x_val, y_val, batch_size=batch_size)
test_complete = SETISequence(x1, batch_size=batch_size)


# Reduced Dataset

In [ ]:
x0 = new_label.index.values
y0 = new_label[target_col].values

x1 = sub.index.values

x_train, x_val, y_train, y_val = train_test_split(x0, y0, test_size=.2, random_state=seed, stratify=y0)

train_reduced = SETISequence(x_train, y_train, batch_size=batch_size)
val_reduced = SETISequence(x_val, y_val, batch_size=batch_size)
test_reduced = SETISequence(x1, batch_size=batch_size)

In [ ]:
def create_model2():
    input_size = (273, 256, 3)
    model = tf.keras.Sequential([
            L.Conv2D(16,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=(273,256,3)),
            L.Conv2D(32,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=(273,256,3)),
            L.Conv2D(64,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=(273,256,3)),
            L.Conv2D(3,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=(273,256,3)),
            efn.EfficientNetB1(input_shape=(273, 256, 3),weights='imagenet',include_top=False),
            L.GlobalAveragePooling2D(),
            L.Dense(1, activation='sigmoid')
            ])

    #model.summary
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-4),
                  loss='binary_crossentropy', metrics=[keras.metrics.AUC(), 'mae', 'accuracy'])
    
    return model

In [ ]:
model2 = create_model2()
model2.summary()

In [ ]:
ckpt = tf.keras.callbacks.ModelCheckpoint(
    "history2.h5", save_best_only=True, use_multiprocessing=True, workers=4, save_weights_only=True,
)

history2 = model2.fit(train_reduced, validation_data=val_reduced, epochs=n_epoch, callbacks=[ckpt])

# Model 2 Performance in reduced dataset

In [ ]:
# Epoch 4: val_loss: 0.4185 - val_auc: 0.9245 - val_mae: 0.1706 - val_accuracy: 0.8530
# Epoch 5: val_loss: 0.3305 - val_auc: 0.9350 - val_mae: 0.1807 - val_accuracy: 0.8653

plot_graphs(history2, 'loss')
plot_graphs(history2, 'auc')
plot_graphs(history2, 'mae')
plot_graphs(history2, 'accuracy')

In [ ]:
y_pred = model2.predict(
    test, use_multiprocessing=True, workers=4, verbose=1
)

sub = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
sub['target'] = y_pred
sub.to_csv('history2.csv', index=False)
sub['target'].hist()

In [ ]:
def create_model3():
    input_size = (273, 256, 3)
    model = tf.keras.Sequential([
            L.Conv2D(3,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=(273,256,3)),
            efn.EfficientNetB1(input_shape=(273, 256, 3),weights='imagenet',include_top=False),
            #L.GlobalAveragePooling2D(),
            #L.Flatten(),
            L.Conv2D(16,(3,3), strides=(1,1), padding="same", activation='relu'),
            L.Conv2D(32,(3,3), strides=(1,1), padding="same", activation='relu'),
            L.Conv2D(64,(3,3), strides=(1,1), padding="same", activation='relu',),
            L.Conv2D(64,(3,3), strides=(1,1), padding="same", activation='relu'),
            L.Conv2D(64,(3,3), strides=(1,1), padding="same", activation='relu'),
            L.Flatten(),
            # 512 neuron hidden layer
            L.Dense(512, activation='relu'),
            L.Dense(1, activation='sigmoid')
            ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-4),
                  loss='binary_crossentropy', metrics=[keras.metrics.AUC(), 'mae', 'accuracy'])
    
    return model

In [ ]:
model3 = create_model3()
model3.summary()

In [ ]:
model3 = create_model3()

ckpt = tf.keras.callbacks.ModelCheckpoint(
    "history3.h5", save_best_only=True, use_multiprocessing=True, workers=4, save_weights_only=True,
)

history3 = model3.fit(train_reduced, validation_data=val_redcued, epochs=5, callbacks=[ckpt])

# Model 3 Performance in reduced dataset

In [ ]:
# Epoch 4: val_loss: 0.3570 - val_auc_4: 0.9291 - val_mae: 0.1731 - val_accuracy: 0.8541
# Epoch 5: val_auc_4: 0.9237 - val_mae: 0.1782 - val_accuracy: 0.8461

plot_graphs(history3, 'loss')
plot_graphs(history3, 'auc_4')
plot_graphs(history3, 'mae')
plot_graphs(history3, 'accuracy')

In [ ]:
y_pred = model3.predict(
    test, use_multiprocessing=True, workers=4, verbose=1
)

sub = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
sub['target'] = y_pred
sub.to_csv('history3.csv', index=False)
sub['target'].hist()

In [ ]:
def create_model4():
    model = tf.keras.Sequential([
            L.Conv2D(3,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=input_size),
            efn.EfficientNetB1(input_shape=input_size,weights='imagenet',include_top=False),
            L.TimeDistributed(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, recurrent_dropout=0.2)), name="bi_gru_1"),
            L.TimeDistributed(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, recurrent_dropout=0.2)), name="bi_gru_2"),
            L.TimeDistributed(L.GlobalAveragePooling1D()),
            L.Dropout(.2),
            L.Flatten(),
            L.Dropout(.2),
            L.Dense(129, activation="sigmoid"),
            L.Dropout(.2),
            L.Dense(66, activation="sigmoid"),
            L.Dropout(.2),
            L.Dense(33, activation="sigmoid"),
            L.Dropout(.2),
            L.Dense(18, activation="sigmoid"),
            L.Dropout(.2),
            L.Dense(9, activation="sigmoid"),
            L.Dense(1, activation="sigmoid"),
            ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-4),
                  loss='binary_crossentropy', metrics=[keras.metrics.AUC(), 'mae', 'accuracy'])
    
    return model

In [ ]:
def create_model5():
    model = tf.keras.Sequential([
            L.Conv2D(3,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=input_size),
            efn.EfficientNetB1(input_shape=input_size,weights='imagenet',include_top=False),
            L.TimeDistributed(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True)), name="bi_gru_1"),
            L.TimeDistributed(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True)), name="bi_gru_2"),
            L.TimeDistributed(L.GlobalAveragePooling1D()),
            L.Dropout(.2),
            L.Flatten(),
            L.Dropout(.2),
            L.Dense(129, activation="sigmoid"),
            L.Dropout(.2),
            L.Dense(66, activation="sigmoid"),
            L.Dropout(.2),
            L.Dense(33, activation="sigmoid"),
            L.Dropout(.2),
            L.Dense(18, activation="sigmoid"),
            L.Dropout(.2),
            L.Dense(9, activation="sigmoid"),
            L.Dense(1, activation="sigmoid"),
            ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-4),
                  loss='binary_crossentropy', metrics=[keras.metrics.AUC(), 'mae', 'accuracy'])
    
    return model

In [ ]:
model4 = create_model4()

ckpt = tf.keras.callbacks.ModelCheckpoint(
    "history4.h5", save_best_only=True, use_multiprocessing=True, workers=4, save_weights_only=True,
)

history4 = model4.fit(train_reduced, validation_data=val_reduced, epochs=5, callbacks=[ckpt])

# Model 4 Performance in reduced dataset

In [ ]:
# Epoch 4: val_loss: 0.6268 - val_auc_6: 0.6879 - val_mae: 0.4273 - val_accuracy: 0.6794
# Epoch 5: val_loss: 0.6949 - val_auc_6: 0.6065 - val_mae: 0.4546 - val_accuracy: 0.5980

plot_graphs(history4, 'loss')
plot_graphs(history4, 'auc_6')
plot_graphs(history4, 'mae')
plot_graphs(history4, 'accuracy')

In [ ]:
y_pred = model4.predict(
    test, use_multiprocessing=True, workers=4, verbose=1
)

sub = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
sub['target'] = y_pred
sub.to_csv('history4.csv', index=False)
sub['target'].hist()

In [ ]:
model5 = create_model5()

ckpt = tf.keras.callbacks.ModelCheckpoint(
    "history5.h5", save_best_only=True, use_multiprocessing=True, workers=4, save_weights_only=True,
)

history5 = model5.fit(train, validation_data=val, epochs=5, callbacks=[ckpt])

# Model 5 Performance in reduced dataset

In [ ]:
# Epoch 4:  val_loss: 0.5886 - val_auc_7: 0.7669 - val_mae: 0.4186 - val_accuracy: 0.7242
# Epoch 5: val_loss: 0.5529 - val_auc_7: 0.7893 - val_mae: 0.3814 - val_accuracy: 0.7572

plot_graphs(history5, 'loss')
plot_graphs(history5, 'auc_7')
plot_graphs(history5, 'mae')
plot_graphs(history5, 'accuracy')

In [ ]:
y_pred = model5.predict(
    test, use_multiprocessing=True, workers=4, verbose=1
)

sub = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
sub['target'] = y_pred
sub.to_csv('history5.csv', index=False)
sub['target'].hist()

In [ ]:
import tensorflow_addons as tfa
loss = tf.keras.losses.BinaryCrossentropy(
                from_logits=False,
                label_smoothing=0,
                reduction='auto',
                name='binary_crossentropy'
    )
    
optimizer = tfa.optimizers.AdamW(
        weight_decay=1e-7,
        learning_rate=1e-4,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=True,
        name='AdamW',
    )

def create_model6():
    input_size = (273, 256, 3)
    model = tf.keras.Sequential([
            L.Conv2D(64,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=(273,256,3)),
            L.MaxPooling2D(2, 2),
            L.Conv2D(32,(3,3), strides=(1,1), padding="same", activation='relu'),
            L.MaxPooling2D(2, 2),
            L.Conv2D(16,(3,3), strides=(1,1), padding="same", activation='relu'),
            L.MaxPooling2D(2, 2),
            L.Conv2D(8,(3,3), strides=(1,1), padding="same", activation='relu'),
            L.MaxPooling2D(2, 2),
            L.Conv2D(3,(3,3), strides=(1,1), padding="same", activation='relu'),
            efn.EfficientNetB1(weights='imagenet',include_top=False),
            L.GlobalAveragePooling2D(),
            L.Dense(1, activation='sigmoid')
            ])

    model.compile(optimizer=optimizer,
                  loss=loss, metrics=[keras.metrics.AUC(), 'mae', 'accuracy'])
    
    return model

def create_model7():
    input_size = (273, 256, 3)
    model = tf.keras.Sequential([
            L.Conv2D(3,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=(273,256,3)),
            efn.EfficientNetB1(weights='imagenet',include_top=False),
            L.GlobalAveragePooling2D(),
            L.Dense(1, activation='sigmoid')
            ])

    model.compile(optimizer=optimizer,
                  loss=loss, metrics=[keras.metrics.AUC(), 'mae', 'accuracy'])
    
    return model


def create_model7_original():
    input_size = (273, 256, 3)
    model = tf.keras.Sequential([
            L.Conv2D(3,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=(273,256,3)),
            efn.EfficientNetB1(weights='imagenet',include_top=False),
            L.GlobalAveragePooling2D(),
            L.Dense(1, activation='sigmoid')
            ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-4),
                  loss='binary_crossentropy', metrics=[keras.metrics.AUC(), 'mae', 'accuracy'])
    
    return model

In [ ]:
scheduler_cb = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    verbose=0,
    mode='auto',
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

model7 = create_model7()
model7_original = create_model7_original()

ckpt = tf.keras.callbacks.ModelCheckpoint(
    "create_model7.h5", save_best_only=True, use_multiprocessing=True, workers=4, save_weights_only=True,
)

In [ ]:
history7_original = model7_original.fit(train_reduced, validation_data=val_reduced, epochs=20, callbacks=[ckpt,scheduler_cb,early_stopping_cb])

# Model 7 Original Performance in reduced dataset

In [ ]:
#Model 7 Base Line SETI Simple Code for Beginners(TensorFlow): 
'''
Data Reduced: New Label
Epoch 5/20
235/235 [==============================] - 119s 506ms/step - loss: 0.1987 - auc_1: 0.9772 - mae: 0.1388 - accuracy: 0.9140 - val_loss: 0.4910 - val_auc_1: 0.8824 - val_mae: 0.2274 - val_accuracy: 0.7977
Epoch 6/20
235/235 [==============================] - 119s 505ms/step - loss: 0.1460 - auc_1: 0.9883 - mae: 0.1022 - accuracy: 0.9406 - val_loss: 0.6574 - val_auc_1: 0.8663 - val_mae: 0.2298 - val_accuracy: 0.7849
Epoch 7/20
235/235 [==============================] - 118s 504ms/step - loss: 0.1084 - auc_1: 0.9935 - mae: 0.0756 - accuracy: 0.9600 - val_loss: 0.6538 - val_auc_1: 0.8736 - val_mae: 0.2174 - val_accuracy: 0.7939
Epoch 8/20
235/235 [==============================] - 120s 509ms/step - loss: 0.0698 - auc_1: 0.9977 - mae: 0.0508 - accuracy: 0.9749 - val_loss: 0.6684 - val_auc_1: 0.8763 - val_mae: 0.2098 - val_accuracy: 0.8009
Epoch 9/20
235/235 [==============================] - 120s 509ms/step - loss: 0.0517 - auc_1: 0.9987 - mae: 0.0379 - accuracy: 0.9867 - val_loss: 0.6982 - val_auc_1: 0.8794 - val_mae: 0.2098 - val_accuracy: 0.7961
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
'''

'''

Epoch 1/20
1255/1255 [==============================] - 775s 603ms/step - loss: 0.2767 - auc_1: 0.7383 - mae: 0.1621 - accuracy: 0.9081 - val_loss: 0.1418 - val_auc_1: 0.9262 - val_mae: 0.0608 - val_accuracy: 0.9599
Epoch 2/20
1255/1255 [==============================] - 773s 615ms/step - loss: 0.1288 - auc_1: 0.9408 - mae: 0.0675 - accuracy: 0.9604 - val_loss: 0.1248 - val_auc_1: 0.9399 - val_mae: 0.0530 - val_accuracy: 0.9651
Epoch 3/20
1255/1255 [==============================] - 781s 622ms/step - loss: 0.0895 - auc_1: 0.9691 - mae: 0.0469 - accuracy: 0.9738 - val_loss: 0.1269 - val_auc_1: 0.9370 - val_mae: 0.0470 - val_accuracy: 0.9665
Epoch 4/20
1255/1255 [==============================] - 773s 615ms/step - loss: 0.0665 - auc_1: 0.9850 - mae: 0.0366 - accuracy: 0.9784 - val_loss: 0.1372 - val_auc_1: 0.9345 - val_mae: 0.0428 - val_accuracy: 0.9663
Epoch 5/20
1255/1255 [==============================] - 771s 614ms/step - loss: 0.0405 - auc_1: 0.9941 - mae: 0.0240 - accuracy: 0.9861 - val_loss: 0.1538 - val_auc_1: 0.9233 - val_mae: 0.0421 - val_accuracy: 0.9667
Epoch 6/20
1255/1255 [==============================] - 776s 618ms/step - loss: 0.0280 - auc_1: 0.9979 - mae: 0.0173 - accuracy: 0.9898 - val_loss: 0.1667 - val_auc_1: 0.9159 - val_mae: 0.0400 - val_accuracy: 0.9647
Epoch 7/20
1255/1255 [==============================] - 773s 616ms/step - loss: 0.0172 - auc_1: 0.9985 - mae: 0.0111 - accuracy: 0.9945 - val_loss: 0.1853 - val_auc_1: 0.9081 - val_mae: 0.0378 - val_accuracy: 0.9659
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
'''

plot_graphs(history7_original, 'loss')
plot_graphs(history7_original, 'auc_1')
plot_graphs(history7_original, 'mae')
plot_graphs(history7_original, 'accuracy')

In [ ]:
y_pred = model.predict(
    test, use_multiprocessing=True, workers=4, verbose=1
)

sub = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
sub['target'] = y_pred
sub.to_csv('seti-breakthrough-listen_sample_submission_model_11_06.csv', index=False)
sub['target'].hist()

In [ ]:
history7 = model7.fit(train_reduced, validation_data=val_reduced, epochs=20, callbacks=[ckpt,scheduler_cb,early_stopping_cb])

# Model 7 Tuned Performance in reduced dataset

In [ ]:
plot_graphs(history7, 'loss')
plot_graphs(history7, 'auc_1')
plot_graphs(history7, 'mae')
plot_graphs(history7, 'accuracy')

In [ ]:
def create_model8():
    input_size = (273, 256, 3)
    model = tf.keras.Sequential([
        L.Conv2D(3,(3,3), strides=(1,1), padding="same", activation='relu', input_shape=input_size),
        efn.EfficientNetB1(input_shape=input_size,weights='imagenet',include_top=False),
    
        L.TimeDistributed(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, recurrent_dropout=0.1)), name="bi_gru_1"),
        #L.Dropout(.2),
        L.TimeDistributed(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, recurrent_dropout=0.1)), name="bi_gru_2"),
        #L.Dropout(.2),
    
        #L.TimeDistributed(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True)), name="bi_gru_1"),
        #L.Dropout(.2),
        
        #L.TimeDistributed(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True)), name="bi_gru_2"),
        #L.Dropout(.2),
        
        L.TimeDistributed(L.GlobalAveragePooling1D()),
        L.Flatten(),
        
        L.Dense(512, activation="sigmoid"),
        L.Dropout(.2),
        L.Dense(256, activation="sigmoid"),
        L.Dropout(.2),
        L.Dense(128, activation="sigmoid"),
        L.Dropout(.1),
        L.Dense(64, activation="sigmoid"),
        L.Dropout(.1),
        L.Dense(1, activation="sigmoid"),
        ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-4),
                  loss='binary_crossentropy', metrics=[keras.metrics.AUC(), 'mae', 'accuracy'])
    
    return model


modelBiGRU.summary()

In [ ]:
model8 = create_model8()

ckpt = tf.keras.callbacks.ModelCheckpoint(
    "create_model8.h5", save_best_only=True, use_multiprocessing=True, workers=4, save_weights_only=True,
)

history8 = model8.fit(train_reduced, validation_data=val_redcued, epochs=5, callbacks=[ckpt])

In [ ]:
plot_graphs(history8, 'loss')
plot_graphs(history8, 'auc')
plot_graphs(history8, 'accuracy')
plot_graphs(history8, 'mae')

# For modelBiGRU best epoch was between 6 and 7 or 14 - 15 -> (ok)
# For modelBiGRU with 7 epochs dropout 0.2 included in GRU -> val_loss: 0.4577 - val_auc_2: 0.9130 (todo)
# For modelBiGRU with 7 epochs dropout 0.2 not included in GRU ->  val_loss: 0.4853 - val_auc_2: 0.9147 (todo)
# For modelBiGRU with 20 epochs dropout 0.2 with dropout between GRU layers -> val_loss: 0.3848 - val_auc_6: 0.9116 - val_mae: 0.1733 - val_accuracy: 0.8637
# For modelBiGRU with 19 epochs dropout 0.2 with dropout between GRU layers -> val_loss: 0.3848 - val_auc_6: 0.9116 - val_mae: 0.1733 - val_accuracy: 0.8637
# For modelBiGRU with 7 epochs dropout 0.2 with dropout between GRU layers -> val_loss: 0.4139 - val_auc_6: 0.8921 - val_mae: 0.2656 - val_accuracy: 0.8253
# For modelBiGRU with 40 epochs dropout 0.2 with dropout between GRU layers -> Stoped in epoch 22: val_loss: 0.5288 - val_auc_10: 0.8153 - val_mae: 0.3297 - val_accuracy: 0.7609
# For modelBiGRU with 20 epochs dropout 0.2 without dropout between GRU layers -> val_loss: 0.3120 - val_auc_7: 0.9250 - val_mae: 0.1711 - val_accuracy: 0.8845 *
# For modelBiGRU with 18 epochs dropout 0.2 without dropout between GRU layers -> val_loss: 0.4750 - val_auc_9: 0.8621 - val_mae: 0.3139 - val_accuracy: 0.7902
# For modelBiGRU with 20 epochs dropout 0.2 without dropout between GRU layers and without drop after Flatten -> val_loss: 0.3904 - val_auc_8: 0.9042 - val_mae: 0.2221 - val_accuracy: 0.8371
# For modelBiGRU with 26 epochs dropout 0.2 without GRU recurrent_dropout -> val_loss: 0.3266 - val_auc: 0.9309 - val_mae: 0.1732 - val_accuracy: 0.8663
# For modelBiGRU with 30 epochs dropout 0.2 without dropout between GRU layers and recurrent_dropout -> val_loss: 0.3781 - val_auc: 0.9121 - val_mae: 0.1751 - val_accuracy: 0.8578
# For modelBiGRU with 50 epochs dropout 0.2 without dropout between GRU layers and recurrent_dropout -> val_loss: 0.3781 - val_auc: 0.9121 - val_mae: 0.1751 - val_accuracy: 0.8578

In [ ]:
y_pred = model8.predict(
    test, use_multiprocessing=True, workers=4, verbose=1
)

sub = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
sub['target'] = y_pred
sub.to_csv('submission_model_50_epochs_with_dropout_gru_and_recurrent_dropout.csv', index=False)
sub['target'].hist()

In [ ]:
sub = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
sub['target'] = y_pred
sub.to_csv('submission_model_50_epochs_with_dropout_gru_and_recurrent_dropout.csv', index=False)
sub['target'].hist()